In [2]:
import os
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType, StringType, DoubleType, DateType
import datetime
from utils.spark_session import spark

base_path = "hello_world"
customers_path = f"{base_path}/customers"
products_path = f"{base_path}/products"
orders_path = f"{base_path}/orders"

# Configurations
num_customers = 5_000_000  
num_products = 5_000_000     
num_orders = 5_000_000       

# ---------------------------------
# Helper: Generate a range DataFrame
# ---------------------------------
def generate_range_df(n, col_name="id"):
    return spark.range(1, n + 1).withColumnRenamed("id", col_name)

# ---------------------------------
# Step 1: Generate customers
# ---------------------------------
print("Generating customers ...")

countries = ["USA", "Canada", "UK", "Germany", "France"]

customers_df = generate_range_df(num_customers, "customer_id") \
    .withColumn("name", f.concat(f.lit("Customer_"), f.col("customer_id").cast(StringType()))) \
    .withColumn("country", f.when(f.rand() < 0.2, countries[0])
                          .when(f.rand() < 0.4, countries[1])
                          .when(f.rand() < 0.6, countries[2])
                          .when(f.rand() < 0.8, countries[3])
                          .otherwise(countries[4])) \
    .withColumn("age", (f.rand() * 52 + 18).cast(IntegerType()))

customers_df.write.mode("overwrite").parquet(customers_path)
print(f"Generated {customers_path}")

# ---------------------------------
# Step 2: Generate products
# ---------------------------------
print("Generating products ...")

categories = ["Electronics", "Clothing", "Books", "Toys", "Sports"]

products_df = generate_range_df(num_products, "product_id") \
    .withColumn("product_name", f.concat(f.lit("Product_"), f.col("product_id").cast(StringType()))) \
    .withColumn("category", f.when(f.rand() < 0.2, categories[0])
                            .when(f.rand() < 0.4, categories[1])
                            .when(f.rand() < 0.6, categories[2])
                            .when(f.rand() < 0.8, categories[3])
                            .otherwise(categories[4])) \
    .withColumn("price", (f.rand() * 495 + 5).cast(DoubleType()))

products_df.write.mode("overwrite").parquet(products_path)
print(f"Generated {products_path}")

# ---------------------------------
# Step 3: Generate orders (Partitioned by year)
# ---------------------------------
print("Generating orders ...")

start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2025, 1, 1)
days_diff = (end_date - start_date).days

orders_df = generate_range_df(num_orders, "order_id") \
    .withColumn("customer_id", (f.rand() * num_customers + 1).cast(IntegerType())) \
    .withColumn("product_id", (f.rand() * num_products + 1).cast(IntegerType())) \
    .withColumn("quantity", (f.rand() * 9 + 1).cast(IntegerType())) \
    .withColumn("price", (f.rand() * 495 + 5).cast(DoubleType())) \
    .withColumn("random_days", (f.rand() * days_diff).cast(IntegerType())) \
    .withColumn("order_date", f.expr(f"date_add('{start_date}', random_days)")) \
    .withColumn("year", f.year("order_date")) \
    .drop("random_days")

# Write as Parquet, partitioned by year
orders_df.write \
    .mode("overwrite") \
    .partitionBy("year") \
    .parquet(orders_path)

print(f"Generated {orders_path}")


Py4JJavaError: An error occurred while calling o180.parquet.
: java.io.IOException: Failed to rename DeprecatedRawLocalFileStatus{path=file:/wsl_mount_point/spark_data/hello_world/customers/_temporary/0/task_202507240050087907389882189297092_0001_m_000000/part-00000-cf88e4e9-e5c3-463c-b0bd-8d8d6c63be3a-c000.snappy.parquet; isDirectory=false; length=12679635; replication=1; blocksize=33554432; modification_time=1753329011627; access_time=1753329011627; owner=; group=; permission=rw-rw-rw-; isSymlink=false; hasAcl=false; isEncrypted=false; isErasureCoded=false} to file:/wsl_mount_point/spark_data/hello_world/customers/part-00000-cf88e4e9-e5c3-463c-b0bd-8d8d6c63be3a-c000.snappy.parquet
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:477)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:490)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.commitJobInternal(FileOutputCommitter

In [ ]:
from utils.spark_session import spark
from utils.display_def import display
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

customers_path = f"hdfs:/{base_path}/customers/part-00000-b98e4cda-ea1a-4351-8341-04ce69be9f9a-c000.snappy.parquet"
orders_path = f"{base_path}/orders"
products_path = f"{base_path}/products"

customers_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("age", IntegerType(), True)
])

customers_df = spark.read.parquet(customers_path)

display(customers_df)

products_schema = StructType([
    StructField("product_id", IntegerType(), False),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True)
])

orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("order_date", DateType(), True),
    StructField("year", IntegerType(), True)
])


orders_df = spark.read.schema(orders_schema).parquet(orders_path)
products_df = spark.read.schema(products_schema).parquet(products_path)


# # Example Transformations
# customers_filtered = customers_df.filter(F.col("country") == "USA")
# orders_enriched = orders_df.withColumn("total_price", F.col("price") * F.col("quantity"))
# orders_products = orders_enriched.join(products_df, on="product_id", how="inner")
# full_data = orders_products.join(customers_filtered, on="customer_id", how="inner")

# customer_spending = (
#     full_data
#     .groupBy("customer_id", "name")
#     .agg(
#         F.sum("total_price").alias("total_spent"),
#         F.countDistinct("order_id").alias("total_orders")
#     )
#     .orderBy(F.desc("total_spent"))
# )

# # Action: Collect top 20 customers by total spending
# top_customers = customer_spending.limit(20)

# # Display results
# display(top_customers)

# # Additional action: Show aggregated sales per product category
# category_sales = (
#     full_data
#     .groupBy("category")
#     .agg(
#         F.sum("total_price").alias("category_sales"),
#         F.countDistinct("order_id").alias("order_count")
#     )
#     .orderBy(F.desc("category_sales"))
# )

# display(category_sales)
